In [1]:
import numpy as np
from gwsnr import GWSNR
from gwsnr import antenna_response_array, cubic_spline_interpolator2d

In [2]:
gwsnr = GWSNR(npool=4, waveform_approximant="IMRPhenomXPHM", duration_max=None, snr_type="ann", pdet=False)

psds not given. Choosing bilby's default psds
Intel processor has trouble allocating memory when the data is huge. So, by default for IMRPhenomXPHM, duration_max = 64.0. Otherwise, set to some max value like duration_max = 600.0 (10 mins)
Interpolator will be loaded for L1 detector from ./interpolator_pickle/L1/partialSNR_dict_0.pickle
Interpolator will be loaded for H1 detector from ./interpolator_pickle/H1/partialSNR_dict_0.pickle
Interpolator will be loaded for V1 detector from ./interpolator_pickle/V1/partialSNR_dict_0.pickle

Chosen GWSNR initialization parameters:

npool:  4
snr type:  ann
waveform approximant:  IMRPhenomXPHM
sampling frequency:  2048.0
minimum frequency (fmin):  20.0
mtot=mass1+mass2
min(mtot):  2.0
max(mtot) (with the given fmin=20.0): 184.98599853446768
detectors:  ['L1', 'H1', 'V1']
psds:  [PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/lertest/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4

/Users/phurailatpamhemantakumar/anaconda3/envs/lertest/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
# set the GW parameters
mass_1 = np.array([5, 10.,50.,200.])
ratio = np.array([1, 0.8,0.5,0.2])
luminosity_distance = np.array([1000, 2000, 3000, 4000])
a_1 = np.array([0.1, 0.2, 0.3, 0.4])
a_2 = np.array([0.1, 0.2, 0.3, 0.4])
tilt_1 = np.array([0.1, 0.2, 0.3, 0.4])
tilt_2 = np.array([0.1, 0.2, 0.3, 0.4])
phi_12 = np.array([0.1, 0.2, 0.3, 0.4])
phi_jl = np.array([0.1, 0.2, 0.3, 0.4])

# pdet calculation with ANN
gwsnr.snr(mass_1=mass_1, mass_2=mass_1*ratio, luminosity_distance=luminosity_distance, a_1=a_1, a_2=a_2, tilt_1=tilt_1, tilt_2=tilt_2, phi_12=phi_12, phi_jl=phi_jl)

{'L1': array([6.64308357, 5.40514946, 9.97871399, 0.        ]),
 'H1': array([3.85338736, 3.0341351 , 6.47034931, 0.        ]),
 'V1': array([2.27271104, 1.77481127, 3.16478086, 0.        ]),
 'optimal_snr_net': array([ 8.00901798,  6.447602  , 12.30674575,  0.        ])}

In [3]:
from gwsnr.utils import (
    dealing_with_psds,
    interpolator_check,
    load_json,
    load_pickle,
    save_pickle,
    save_json,
    load_ann_h5_from_module,
    load_ann_h5,
    load_pickle_from_module,
    load_json_from_module,
)

In [4]:
# load the ANN models
modelL1 = load_ann_h5('../ann_modelL1_final.h5')
modelH1 = load_ann_h5('../ann_modelH1_final.h5')
modelV1 = load_ann_h5('../ann_modelV1_final.h5')

# load the feature scaler
scalerL1 = load_pickle('../scalerL1_final.pkl')
scalerH1 = load_pickle('../scalerH1_final.pkl')
scalerV1 = load_pickle('../scalerV1_final.pkl')

# load the correction slope and intercept
correctionL1 = load_json('../error_adjustmentL1_final.json')
correctionH1 = load_json('../error_adjustmentH1_final.json')
correctionV1 = load_json('../error_adjustmentV1_final.json')
aL1 = correctionL1['slope']
bL1 = correctionL1['intercept']
aH1 = correctionH1['slope']
bH1 = correctionH1['intercept']
aV1 = correctionV1['slope']
bV1 = correctionV1['intercept']

In [5]:
unlensed_params = load_json("../ler_data/unlensed_param_testing.json")

In [6]:
def input_output(idx, params):
    """
        Function to generate input and output data for the neural network

        Parameters:
        idx: index of the parameter points
        params: dictionary of parameter points
            params.keys() = ['mass_1', 'mass_2', 'luminosity_distance', 'theta_jn', 'psi', 'geocent_time', 'ra', 'dec', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'L1']

        Returns:
        X: input data, [snr_half_[0], amp0[0], eta, chi_eff, theta_jn]
        y: output data, [L1]
    """

    mass_1 = np.array(params['mass_1'])[idx]
    mass_2 = np.array(params['mass_2'])[idx]
    luminosity_distance = np.array(params['luminosity_distance'])[idx]
    theta_jn = np.array(params['theta_jn'])[idx]
    psi = np.array(params['psi'])[idx]
    geocent_time = np.array(params['geocent_time'])[idx]
    ra = np.array(params['ra'])[idx]
    dec = np.array(params['dec'])[idx]
    
    detector_tensor = gwsnr.detector_tensor_list
    snr_halfscaled = np.array(gwsnr.snr_partialsacaled_list)
    ratio_arr = gwsnr.ratio_arr
    mtot_arr = gwsnr.mtot_arr
    
    size = len(mass_1)
    len_ = len(detector_tensor)
    mtot = mass_1 + mass_2
    ratio = mass_2 / mass_1
    # get array of antenna response
    Fp, Fc = antenna_response_array(ra, dec, geocent_time, psi, detector_tensor)

    Mc = ((mass_1 * mass_2) ** (3 / 5)) / ((mass_1 + mass_2) ** (1 / 5))
    eta = mass_1 * mass_2/(mass_1 + mass_2)**2.
    A1 = Mc ** (5.0 / 6.0)
    ci_2 = np.cos(theta_jn) ** 2
    ci_param = ((1 + np.cos(theta_jn) ** 2) / 2) ** 2
    
    size = len(mass_1)
    snr_half_ = np.zeros((len_,size))
    d_eff = np.zeros((len_,size))

    # loop over the detectors
    for j in range(len_):
        # loop over the parameter points
        for i in range(size):
            snr_half_coeff = snr_halfscaled[j]
            snr_half_[j,i] = cubic_spline_interpolator2d(mtot[i], ratio[i], snr_half_coeff, mtot_arr, ratio_arr)
            d_eff[j,i] =luminosity_distance[i] / np.sqrt(
                    Fp[j,i]**2 * ci_param[i] + Fc[j,i]**2 * ci_2[i]
                )

    #amp0
    amp0 =  A1 / d_eff

    # get spin parameters
    a_1 = np.array(params['a_1'])[idx]
    a_2 = np.array(params['a_2'])[idx]
    tilt_1 = np.array(params['tilt_1'])[idx]
    tilt_2 = np.array(params['tilt_2'])[idx]

    # effective spin
    chi_eff = (mass_1 * a_1 * np.cos(tilt_1) + mass_2 * a_2 * np.cos(tilt_2)) / (mass_1 + mass_2)


    # input data
    XL1 = np.vstack([snr_half_[0], amp0[0], eta, chi_eff, theta_jn]).T
    XH1 = np.vstack([snr_half_[1], amp0[1], eta, chi_eff, theta_jn]).T
    XV1 = np.vstack([snr_half_[2], amp0[2], eta, chi_eff, theta_jn]).T

    # output data
    # get snr for y train
    yL1 = np.array(params['L1'])[idx]
    yH1 = np.array(params['H1'])[idx]
    yV1 = np.array(params['V1'])[idx]
    yNET = np.sqrt(yL1**2 + yH1**2 + yV1**2)

    return(XL1, yL1, XH1, yH1, XV1, yV1, yNET)

In [7]:
XL1, yL1, XH1, yH1, XV1, yV1, yNET = input_output(np.arange(len(unlensed_params['optimal_snr_net'])), unlensed_params)

/Users/phurailatpamhemantakumar/anaconda3/envs/lertest/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'detector_tensor' of function 'antenna_response_array'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../gwsnr/njit_functions.py", line 247:
@njit
def antenna_response_array(ra, dec, time, psi, detector_tensor):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


In [10]:
def predict_snr_net_ANN(XL1, XH1, XV1):
    """
        Function to predict the network SNR using the ANN models

        Parameters:
        XL1: input data for L1, [snr_half_[0], amp0[0], eta, chi_eff, theta_jn]
        XH1: input data for H1, [snr_half_[1], amp0[1], eta, chi_eff, theta_jn]
        XV1: input data for V1, [snr_half_[2], amp0[2], eta, chi_eff, theta_jn]

        Returns:
        y: network SNR
    """
    x = scalerL1.transform(XL1)
    yL1 = modelL1.predict(x)
    yL1 = yL1 - (aL1*yL1 + bL1)
    x = scalerH1.transform(XH1)
    yH1 = modelH1.predict(x)
    yH1 = yH1 - (aH1*yH1 + bH1)
    x = scalerV1.transform(XV1)
    yV1 = modelV1.predict(x)
    yV1 = yV1 - (aV1*yV1 + bV1)
    y = np.sqrt(yL1**2 + yH1**2 + yV1**2)

    mass_1 = np.array(unlensed_params['mass_1'])
    mass_2 = np.array(unlensed_params['mass_2'])
    mtot = mass_1 + mass_2
    y[mtot>gwsnr.mtot_max] = 0.

    return(y)

In [11]:
predict_snr_net_ANN(XL1, XH1, XV1).flatten()

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step


array([1.3222325, 1.9176006, 1.1828067, ..., 1.2374389, 2.1879537,
       1.3156186], dtype=float32)

In [12]:
# left: predicted snr, right: actual snr
y_pred = predict_snr_net_ANN(XL1, XH1, XV1).flatten()
idx = (y_pred>4) & (y_pred<10)
y_pred_ = y_pred[idx]
y_true_ = yNET[idx]
print(np.concatenate((y_pred_.reshape(len(y_pred_),1), y_true_.reshape(len(y_true_),1)),1))

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 247us/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 224us/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step
[[4.62136269 4.74315895]
 [4.31896448 4.41793194]
 [5.00712204 5.29411108]
 ...
 [5.46714211 5.69066226]
 [4.16106319 3.79827401]
 [4.21725225 4.43522929]]


In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred_ = predict_snr_net_ANN(XL1, XH1, XV1).flatten()
y_test_ = yNET
cm = confusion_matrix((y_test_>8), ((y_pred_)>8))
print(cm)
accuracy = accuracy_score((y_test_>8), (y_pred_>8))*100
print(f"Accuracy: {accuracy:.3f}%")

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 224us/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 348us/step
[[49723    28]
 [   24   225]]
Accuracy: 99.896%


In [22]:
snr_gwsnr = gwsnr.snr(gw_param_dict=unlensed_params)['optimal_snr_net']
snr_bilby = np.array(unlensed_params['optimal_snr_net'])

In [24]:
y_pred = snr_gwsnr
idx = (y_pred>4) & (y_pred<10)
y_pred_ = y_pred[idx]
y_true_ = snr_bilby[idx]
print(np.concatenate((y_pred_.reshape(len(y_pred_),1), y_true_.reshape(len(y_true_),1)),1))

[[4.61133845 4.74315895]
 [4.33200715 4.41793194]
 [4.89267168 5.29411108]
 ...
 [5.43964279 5.69066226]
 [4.22240297 3.79827401]
 [4.16183341 4.43522929]]


In [23]:
y_pred_ = snr_gwsnr
y_test_ = snr_bilby
cm = confusion_matrix((y_test_>8), ((y_pred_)>8))
print(cm)
accuracy = accuracy_score((y_test_>8), (y_pred_>8))*100
print(f"Accuracy: {accuracy:.3f}%")

[[49722    29]
 [   28   221]]
Accuracy: 99.886%


In [3]:
import tensorflow as tf

# List all physical devices
devices = tf.config.list_physical_devices()
print("Available devices:")
for device in devices:
    print(device)

Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
